In [1]:
using PyPlot

/Users/kyle/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


## Motion Primitive representation

### arc length (direct function of velocity), radius of curvature

In [2]:
# Return a library of motion primitives (arcs of constant radius) representing short paths that the car can follow.
function MotionPrimitives(v_min::Float64=0, 
    v_max::Float64=99, 
    v_steps::Int=100, 
    δ_max::Float64=Float64(π)/4, # max steering angle
    δ_steps::Int=20, # resolution of steering angle
    Δt::Float64=.05, # size of time-steps
    h::Int=10 # lookahead horizon (in time-steps)
    )
    
    car_length = 2.5 # front wheel to back wheel

    # 3D array to store motion_primitives
    motion_primitives = zeros(v_steps,2*δ_steps+1,2) # v: 0,...,99; δ: -45:45, (arc length, +/- radius)

    v = linspace(v_min,v_max,v_steps)*ones(1,2*δ_steps+1)
    δ = (linspace(-δ_max,δ_max,δ_steps*2+1)*ones(1,v_steps))' # steering angle

    motion_primitives[:,:,1] = v*Δt*h # arc length = velocity * time
    motion_primitives[:,:,2] = car_length./sin(δ) # radius of curvature (+ or -)
    motion_primitives[:,1+δ_steps,2] = Inf; # radius of curvature is infinite if steering angle = 0
    
    destination_primitives = zeros(v_steps,2*δ_steps+1,h,3) # lookup table defining car's location at each of next h time steps

    for i = 1:h
        # angle = 2π * arc_length / r
        dθ = v*Δt*i ./ abs(motion_primitives[:,:,2])
        
        # dX = radius * (1 - cos(angle))
        destination_primitives[:,:,i,1] = motion_primitives[:,:,2].*(1 - cos(dθ))
        destination_primitives[:,1+δ_steps,i,1] = 0 # centerline
        
        # dY = abs(radius) * sin(angle)
        destination_primitives[:,:,i,2] = abs(motion_primitives[:,:,2]) .* sin(dθ)
        destination_primitives[:,1+δ_steps,i,2] = v[:,1+δ_steps]*Δt*i # centerline
        
        destination_primitives[:,:,i,3] = dθ
    end
    
    # motion_primitives[v, δ, 1,2] = s (arc_length), r (radius of curvature)
    # destination_primitives are index by [v, δ, h, 1,2,3] = dx, dy, dθ = changes in x, y and θ after h time steps
    return motion_primitives, destination_primitives

end

MotionPrimitives (generic function with 8 methods)

In [19]:
# quadratic logical mask
function QuadraticMask!(library)
    A = size(library)[1]
    B = size(library)[2]
    X = linspace(0,99,A)*ones(1,B)
    Y = ones(A,1)*linspace(-20,20,B)'
    f = 24*Y.^2 + X.^2
    mask = 1.0*(f.<10000)
    
#     fig = figure()
#     ax = gca()
#     p = scatter(X,Y.*mask)
#     xlabel("velocity")
#     ylabel("steering angle")
#     fig[:canvas][:draw]()

    if length(size(library)) == 3
        for i in 1 : size(library)[3]
            library[:,:,i] = library[:,:,i] .* mask
        end
    end
    if length(size(library)) == 4
        for i in 1 : size(library)[3]
            for j in 1 : size(library)[4]
                library[:,:,i,j] = library[:,:,i,j] .* mask
            end
        end
    end
end

QuadraticMask! (generic function with 3 methods)

In [20]:
v_min=0.0 # m/s
v_max=20.0 # m/s
v_steps=20 
δ_max=Float64(π)/8 # max steering angle - radians
δ_steps=12 # resolution of steering angle
Δt=.05 # size of time-steps
h=10 # lookahead horizon (in time-steps)

# get motion primitives library
arc_angle_library, x_y_θ_library = MotionPrimitives(v_min,v_max,v_steps,δ_max,δ_steps,Δt,h);

# apply logical mask to motion primitives library - eliminate impossible turns at high speeds
QuadraticMask!(arc_angle_library)
QuadraticMask!(x_y_θ_library)

# x_y_θ_library[:,:,10,3]
dx = x_y_θ_library[5,5,10,1]
dy = x_y_θ_library[5,5,10,2]
dθ = x_y_θ_library[5,5,10,3]
dx,dy,dθ

(-0.22851731806354408,2.0886348222661764,0.21795288008633326)